In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np

import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from fastai.text.all import *
import fastai
import re

print(torch.__version__)
print(transformers.__version__)
print( fastai.__version__)

1.12.1+cu113
4.22.2
2.7.9


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/데이터/웹툰 생성/webtoon_final.csv')
df10 = pd.read_csv('/content/drive/MyDrive/데이터/웹툰 생성/네이버_도전웹툰_판타지_정제.csv')

In [ ]:
df1.head(2)

,title,story
0,참교육,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다
1,신의 탑,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년 그리고 그런 소년을 시험하는 탑


In [ ]:
df10.drop(['Unnamed: 0', 'id','author'], axis=1, inplace = True)

In [ ]:
df = pd.concat([df1,df10])

In [ ]:
df.shape

(5669, 2)

In [ ]:
df.reset_index(inplace=True, drop = True)

In [ ]:
df.dropna(axis = 0, inplace = True)

## 데이터 전처리

In [ ]:
df.drop_duplicates(['story'], inplace = True)

In [ ]:
# 부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 A-Z a-z 0-9 ]', ' ', text)

In [ ]:
df['story']=df['story'].apply(alpha_num)

In [ ]:
# 중복 공백을 제거해주는 함수
def space(text):
  return re.sub(r"\s+", " ", text)

In [ ]:
df['story']=df['story'].apply(space)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/데이터/웹툰 생성/final_data0929.csv')

In [ ]:
story = []
# 데이터프레임 값들을 리스트로 저장
story.extend(list(df.story.values)) 
story[:5]

['무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다 ',
 '자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년 그리고 그런 소년을 시험하는 탑',
 '여자도 군대에 간다면 본격 여자도 군대 가는 만화',
 '혼자서 자전거를 즐겨타던 모범생 조자현 원치 않게 자전거 크루의 일에 자꾸 휘말리게 되는데 자유를 꿈꾸는 청춘들의 스트릿라이딩 드라마',
 '외모지상주의 싸움독학 인생존망 과 세계관을 공유하는 작품 공부 싸움 외모 뭐 하나 잘난 것 없는 평범한 고등학생 김수현 의 눈앞에 갑자기 퀘스트창이 나타났다 첫 번째 퀘스트 엄마한테 사과하기 를 완료한 수현은 보상으로 키가 3cm 커지게 되는데 퀘스트 엄청 쉽잖아 어떤 퀘스트든 완료해주마 그런데 다음 퀘스트가 system 학교 일진녀와 키스하십시오']

In [ ]:
lines=" ".join(story)
len(lines)

395498

## 모델링

In [ ]:
#download model and tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 

model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")

In [ ]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

train=lines[:int(len(lines)*0.9)]
test=lines[int(len(lines)*0.9):]
splits = [[0],[1]]

tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
batch,seq_len = 8, 256 
dls = tls.dataloaders(bs=batch, seq_len=seq_len)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]
        
        
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()
lr=learn.lr_find()
print(lr)
learn.fit_one_cycle(5, lr)

In [ ]:
torch.save(learn.model, '/content/drive/MyDrive/데이터/웹툰 생성/webtoon_final_model0929(2).pt')

## 결과 

In [ ]:
prompt="첫사랑"
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = learn.model.generate(inp,
                           max_length=60,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 
tokenizer.decode(preds[0].cpu().numpy())

'첫사랑과 이별 후 10년이 지난 지금 그녀는 과연 행복해질 수 있을까  어느 날 갑자기 나타난 수상한 남자 그리고 그 앞에 등장한 한 소녀의 이야기 당신은 누구일까? 나는 왜 이 여자하고 결혼하지 않나요 내게는 특별한 사랑 고백으로 시작된 두 사람의 로맨스 판타지'